# Mettre en place MLflow

In [25]:
# Install the following librairies (it is better to create a venv (or conda) virtual environment first and install these librairies in it)
!pip install mlflow
!pip install --upgrade jinja2
!pip install --upgrade Flask
!pip install setuptools

In [26]:

# starts an MLflow server locally.
!mlflow server --host 127.0.0.1 --port 5000


^C


Initier une nouvelle Expérience.

Démarrer des Exécutions au sein d'une Expérience.

Documenter les paramètres, les métriques et les balises pour vos Exécutions.

Enregistrer des artefacts liés aux exécutions, tels que des modèles, des tableaux, des graphiques, et plus encore.


In [5]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor


In [6]:
# In order to connect to the tracking server, we’ll need to use the uri that we assigned the server when we started it.

client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

#it allows programmatic interaction with the MLflow tracking server.

Nous avons maintenant une interface client vers le serveur de suivi qui peut à la fois envoyer des données au serveur de suivi et en récupérer.



In [7]:
all_experiments = client.search_experiments()

print(all_experiments)


[<Experiment: artifact_location='mlflow-artifacts:/286315274138648484', creation_time=1725693195773, experiment_id='286315274138648484', last_update_time=1725693195773, lifecycle_stage='active', name='LogisticRegression_Models', tags={}>, <Experiment: artifact_location='mlflow-artifacts:/892581525100417762', creation_time=1725647809507, experiment_id='892581525100417762', last_update_time=1725647809507, lifecycle_stage='active', name='Apple_Models', tags={'mlflow.note.content': 'This is the grocery forecasting project. This '
                        'experiment contains the produce models for apples.',
 'project_name': 'grocery-forecasting',
 'project_quarter': 'Q3-2023',
 'store_dept': 'produce',
 'team': 'stores-ml'}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1725646407256, experiment_id='0', last_update_time=1725646407256, lifecycle_stage='active', name='Default', tags={}>]


### Créer une expérience

In [8]:
# Fournir une description de l'expérience qui apparaîtra dans l'interface utilisateur
experiment_description = (
    "Ceci est un projet de prédiction de défaut de paiement des clients. "
    "Cette expérience contient les modèles pour la prédiction de défaut de crédit."
)

# Fournir des balises (tags) recherchables qui définissent les caractéristiques des exécutions (Runs)
# qui feront partie de cette expérience
experiment_tags = {
    "project_name": "credit-default-prediction",
    "business_unit": "risk-management",
    "team": "data-science",
    "project_quarter": "Q3-2023",
    "mlflow.note.content": experiment_description,
}

# Créer l'expérience en fournissant un nom unique
credit_default_experiment = client.create_experiment(
    name="Credit_Default_Models", tags=experiment_tags
)

In [9]:

# Utiliser search_experiments() pour rechercher les expériences par la balise project_name

credit_default_experiment = client.search_experiments(
    filter_string="tags.`project_name` = 'credit-default-prediction'"
)

print(vars(credit_default_experiment[0]))



{'_experiment_id': '544091467554629469', '_name': 'Credit_Default_Models', '_artifact_location': 'mlflow-artifacts:/544091467554629469', '_lifecycle_stage': 'active', '_tags': {'business_unit': 'risk-management', 'mlflow.note.content': 'Ceci est un projet de prédiction de défaut de paiement des clients. Cette expérience contient les modèles pour la prédiction de défaut de crédit.', 'project_name': 'credit-default-prediction', 'project_quarter': 'Q3-2023', 'team': 'data-science'}, '_creation_time': 1725784122891, '_last_update_time': 1725784122891}


### Dataset

In [10]:
import pandas as pd

# Charger les données
data = pd.read_csv("Loan_Data(1).csv")



In [11]:

data.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


### Logging our first runs with MLflow

In [12]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [13]:
# This function call sets the global tracking URI for the current session.
# It’s a convenient way to configure the tracking server URI without creating a separate client instance.

mlflow.set_tracking_uri("http://127.0.0.1:5000")


In [15]:
# Sets the current active experiment to the "Apple_Models" experiment and
# returns the Experiment metadata
# Définir l'expérience active pour votre projet
customer_default_experiment = mlflow.set_experiment("Client_Default_Prediction")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "default_rf_test"

# Define an artifact path that the model will be saved to.
# Définir le chemin des artefacts
artifact_path = "rf_default_model"



### Modèle

In [27]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Définir l'URI de suivi
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Créer ou sélectionner l'expérience pour la régression logistique
mlflow.set_experiment("LogisticRegression_Models")

# Définir plusieurs combinaisons de paramètres à tester
params_list = [

    {"C": 0.01, "solver": 'liblinear', "random_state": 42, "class_weight": 'balanced'},
    {"C": 0.1, "solver": 'liblinear', "random_state": 42, "class_weight": 'balanced'},
    {"C": 1.0, "solver": 'liblinear', "random_state": 42, "class_weight": 'balanced'},
    {"C": 10.0, "solver": 'liblinear', "random_state": 42, "class_weight": 'balanced'},
    {"C": 100.0, "solver": 'liblinear', "random_state": 42, "class_weight": 'balanced'},
    {"C": 1.0, "solver": 'lbfgs', "random_state": 42, "class_weight": 'balanced', "max_iter": 100},
    {"C": 1.0, "solver": 'lbfgs', "random_state": 42, "class_weight": 'balanced', "max_iter": 200},
    {"C": 1.0, "solver": 'saga', "random_state": 42, "class_weight": 'balanced', "max_iter": 100},
    {"C": 1.0, "solver": 'saga', "random_state": 42, "class_weight": 'balanced', "max_iter": 200},
    {"C": 0.5, "solver": 'saga', "random_state": 42, "class_weight": 'balanced', "penalty": 'l1'},
    {"C": 0.5, "solver": 'saga', "random_state": 42, "class_weight": 'balanced', "penalty": 'l2'},
    {"C": 0.01, "solver": 'lbfgs', "random_state": 42, "class_weight": 'balanced', "max_iter": 1000},
]


# Charger vos données

X = data.drop(columns=["customer_id", "default"])
y = data["default"]

# Diviser les données
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Boucle pour entraîner le modèle avec différentes combinaisons de paramètres
for i, params in enumerate(params_list):
    # Démarrer un run pour chaque ensemble de paramètres
    with mlflow.start_run(run_name=f"logistic_regression_run_{i+1}"):
        # Entraîner le modèle de régression logistique
        lr_model = LogisticRegression(**params)
        lr_model.fit(X_train, y_train)

        # Prédire sur l'ensemble de validation
        y_pred = lr_model.predict(X_val)

        # Calculer les métriques de classification
        accuracy = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)

        # Assembler les métriques dans une collection pour les enregistrer
        metrics = {"accuracy": accuracy, "f1_score": f1, "precision": precision, "recall": recall}

        # Enregistrer les paramètres et les métriques dans MLflow
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)

        # Enregistrer le modèle entraîné comme artefact
        mlflow.sklearn.log_model(
            sk_model=lr_model, input_example=X_val, artifact_path="logistic_regression_model"
        )

        # Afficher le rapport de classification
        print(f"Run {i+1} completed with parameters: {params}")
        print(classification_report(y_val, y_pred))


c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

Run 1 completed with parameters: {'C': 0.01, 'solver': 'liblinear', 'random_state': 42, 'class_weight': 'balanced'}
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1652
           1       0.91      0.98      0.94       348

    accuracy                           0.98      2000
   macro avg       0.95      0.98      0.96      2000
weighted avg       0.98      0.98      0.98      2000



c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

Run 2 completed with parameters: {'C': 0.1, 'solver': 'liblinear', 'random_state': 42, 'class_weight': 'balanced'}
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1652
           1       0.91      0.98      0.94       348

    accuracy                           0.98      2000
   macro avg       0.95      0.98      0.96      2000
weighted avg       0.98      0.98      0.98      2000



c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

...

Run 10 completed with parameters: {'C': 0.5, 'solver': 'saga', 'random_state': 42, 'class_weight': 'balanced', 'penalty': 'l1'}
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1652
           1       0.91      0.98      0.94       348

    accuracy                           0.98      2000
   macro avg       0.95      0.98      0.96      2000
weighted avg       0.98      0.98      0.98      2000



c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c

Run 11 completed with parameters: {'C': 0.5, 'solver': 'saga', 'random_state': 42, 'class_weight': 'balanced', 'penalty': 'l2'}
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1652
           1       0.91      0.98      0.94       348

    accuracy                           0.98      2000
   macro avg       0.95      0.98      0.96      2000
weighted avg       0.98      0.98      0.98      2000



c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

Run 12 completed with parameters: {'C': 0.01, 'solver': 'lbfgs', 'random_state': 42, 'class_weight': 'balanced', 'max_iter': 1000}
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1652
           1       0.92      0.99      0.95       348

    accuracy                           0.98      2000
   macro avg       0.96      0.99      0.97      2000
weighted avg       0.98      0.98      0.98      2000



In [28]:
import pandas as pd
import numpy as np
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pickle

# Diviser les données en caractéristiques (X) et cible (y)
X = data.drop(columns=["customer_id", "default"])  # Enlever les colonnes 'customer_id' et 'default'
y = data["default"]

# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir une nouvelle expérience pour le modèle Random Forest
mlflow.set_experiment("RandomForest_Models")

# Liste des ensembles de paramètres à explorer
params_list = [
    {"n_estimators": 50, "max_depth": 4, "min_samples_split": 5, "min_samples_leaf": 2, "bootstrap": True, "oob_score": True, "random_state": 888, "class_weight": 'balanced'},
    {"n_estimators": 100, "max_depth": 6, "min_samples_split": 10, "min_samples_leaf": 4, "bootstrap": True, "oob_score": True, "random_state": 888, "class_weight": 'balanced'},
    {"n_estimators": 200, "max_depth": 8, "min_samples_split": 15, "min_samples_leaf": 6, "bootstrap": True, "oob_score": True, "random_state": 888, "class_weight": 'balanced'},
    {"n_estimators": 100, "max_depth": None, "min_samples_split": 10, "min_samples_leaf": 2, "bootstrap": True, "oob_score": True, "random_state": 888, "class_weight": 'balanced'},
    {"n_estimators": 150, "max_depth": 10, "min_samples_split": 8, "min_samples_leaf": 3, "bootstrap": True, "oob_score": True, "random_state": 888, "class_weight": 'balanced'},
    {"n_estimators": 200, "max_depth": None, "min_samples_split": 5, "min_samples_leaf": 1, "bootstrap": False, "oob_score": False, "random_state": 888, "class_weight": 'balanced'},
]

best_rf_model = None  # Variable pour stocker le meilleur modèle

# Itérer sur chaque ensemble de paramètres et effectuer un run pour chaque configuration
for params in params_list:
    with mlflow.start_run(run_name=f"random_forest_run_{params['n_estimators']}_{params['max_depth']}"):
        # Entraîner le modèle RandomForestClassifier avec les paramètres actuels
        rf = RandomForestClassifier(**params)
        rf.fit(X_train, y_train)

        # Prédire sur l'ensemble de validation
        y_pred = rf.predict(X_val)

        # Calculer les métriques de classification
        accuracy = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)

        # Assembler les métriques dans une collection pour les enregistrer
        metrics = {"accuracy": accuracy, "f1_score": f1, "precision": precision, "recall": recall}

        # Enregistrer les paramètres et les métriques dans MLflow
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)

        # Enregistrer une instance du modèle entraîné pour une utilisation ultérieure dans MLflow
        mlflow.sklearn.log_model(
            sk_model=rf, input_example=X_val, artifact_path="random_forest_model"
        )

        # Afficher le rapport de classification
        print(f"Run completed with parameters: {params}")
        print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

        # Mettre à jour le meilleur modèle
        best_rf_model = rf

# Enregistrer le dernier modèle (considéré comme le meilleur modèle) au format pickle
if best_rf_model is not None:
    with open("random_forest_model2pkl", "wb") as f:
        pickle.dump(best_rf_model, f)

c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

Run completed with parameters: {'n_estimators': 50, 'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 2, 'bootstrap': True, 'oob_score': True, 'random_state': 888, 'class_weight': 'balanced'}
Accuracy: 0.9725, F1-Score: 0.9262, Precision: 0.8690, Recall: 0.9914


c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

Run completed with parameters: {'n_estimators': 100, 'max_depth': 6, 'min_samples_split': 10, 'min_samples_leaf': 4, 'bootstrap': True, 'oob_score': True, 'random_state': 888, 'class_weight': 'balanced'}
Accuracy: 0.9915, F1-Score: 0.9759, Precision: 0.9636, Recall: 0.9885


c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

...

Run completed with parameters: {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 2, 'bootstrap': True, 'oob_score': True, 'random_state': 888, 'class_weight': 'balanced'}
Accuracy: 0.9940, F1-Score: 0.9828, Precision: 0.9828, Recall: 0.9828


c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

Run completed with parameters: {'n_estimators': 150, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 3, 'bootstrap': True, 'oob_score': True, 'random_state': 888, 'class_weight': 'balanced'}
Accuracy: 0.9925, F1-Score: 0.9785, Precision: 0.9744, Recall: 0.9828


c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

Run completed with parameters: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1, 'bootstrap': False, 'oob_score': False, 'random_state': 888, 'class_weight': 'balanced'}
Accuracy: 0.9955, F1-Score: 0.9870, Precision: 0.9913, Recall: 0.9828


#### modèle pour faire le versionning comme demandé

In [30]:
import pandas as pd
import numpy as np
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pickle

# Diviser les données en caractéristiques (X) et cible (y)
X = data.drop(columns=["customer_id", "default"])  # Enlever les colonnes 'customer_id' et 'default'
y = data["default"]

# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir une nouvelle expérience pour le modèle Random Forest
mlflow.set_experiment("RandomForest_Models")

# Définir un ensemble de paramètres pour le modèle RandomForestClassifier
params = {
    "n_estimators": 100,
    "max_depth": 5,
    "min_samples_split": 20,
    "min_samples_leaf": 10,
    "bootstrap": True,
    "oob_score": True,
    "random_state": 888,
    "class_weight": 'balanced'
}

# Démarrer un run dans MLflow
with mlflow.start_run(run_name=f"random_forest_run_{params['n_estimators']}_{params['max_depth']}"):
    # Entraîner le modèle RandomForestClassifier avec les paramètres actuels
    rf = RandomForestClassifier(**params)
    rf.fit(X_train, y_train)

    # Prédire sur l'ensemble de validation
    y_pred = rf.predict(X_val)

    # Calculer les métriques de classification
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    # Assembler les métriques dans une collection pour les enregistrer
    metrics = {"accuracy": accuracy, "f1_score": f1, "precision": precision, "recall": recall}

    # Enregistrer les paramètres et les métriques dans MLflow
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)

    # Enregistrer une instance du modèle entraîné pour une utilisation ultérieure dans MLflow
    mlflow.sklearn.log_model(
        sk_model=rf, input_example=X_val, artifact_path="random_forest_model"
    )

    # Afficher le rapport de classification
    print(f"Run completed with parameters: {params}")
    print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")




c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

Run completed with parameters: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 20, 'min_samples_leaf': 10, 'bootstrap': True, 'oob_score': True, 'random_state': 888, 'class_weight': 'balanced'}
Accuracy: 0.9825, F1-Score: 0.9517, Precision: 0.9151, Recall: 0.9914


In [31]:
import pandas as pd
import numpy as np
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pickle

# Diviser les données en caractéristiques (X) et cible (y)
X = data.drop(columns=["customer_id", "default"])  # Enlever les colonnes 'customer_id' et 'default'
y = data["default"]

# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir une nouvelle expérience pour le modèle de Régression Logistique
mlflow.set_experiment("LogisticRegression_Models")

# Définir un ensemble de paramètres pour le modèle LogisticRegression
params = {
    "C": 1.0,
    "solver": 'liblinear',
    "random_state": 42,
    "class_weight": 'balanced'
}

# Démarrer un run dans MLflow
with mlflow.start_run(run_name=f"logistic_regression_run_{params['C']}"):
    # Entraîner le modèle LogisticRegression avec les paramètres actuels
    lr = LogisticRegression(**params)
    lr.fit(X_train, y_train)

    # Prédire sur l'ensemble de validation
    y_pred = lr.predict(X_val)

    # Calculer les métriques de classification
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    # Assembler les métriques dans une collection pour les enregistrer
    metrics = {"accuracy": accuracy, "f1_score": f1, "precision": precision, "recall": recall}

    # Enregistrer les paramètres et les métriques dans MLflow
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)

    # Enregistrer une instance du modèle entraîné pour une utilisation ultérieure dans MLflow
    mlflow.sklearn.log_model(
        sk_model=lr, input_example=X_val, artifact_path="logistic_regression_model"
    )

    # Afficher le rapport de classification
    print(f"Run completed with parameters: {params}")
    print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

# Enregistrer le modèle au format pickle
with open("logistic_regression_model.pkl", "wb") as f:
    pickle.dump(lr, f)


c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\alban\OneDrive\Documents\MLolp\projet-main\.venv\Lib\site-packages\mlflow\types\utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent mis

Run completed with parameters: {'C': 1.0, 'solver': 'liblinear', 'random_state': 42, 'class_weight': 'balanced'}
Accuracy: 0.9790, F1-Score: 0.9420, Precision: 0.9069, Recall: 0.9799
